<a href="https://colab.research.google.com/github/SaiPraneeth-E/brain-tumor-app/blob/main/BrainTumour.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================================================================
#                 FULL FIXED: Brain Tumor Classification (Colab-ready)
# ==============================================================================
# Paste this into a Google Colab cell (or save as a .py and run in a suitable env).
# Main fixes:
#  - Robust detection of the last convolutional layer for Grad-CAM (avoids AttributeError)
#  - Safer filesystem operations (no shell magics required)
#  - Reliable conversion of markdown-style **bold** to HTML using regex
#  - Minor robustness improvements and helpful error messages

print("PART 0: Authenticating with Kaggle API...")

import os
import sys
import re
import zipfile
import glob
import io
import shutil
import subprocess

# Colab-specific utilities (if running in Colab)
try:
    from google.colab import files
    IN_COLAB = True
except Exception:
    IN_COLAB = False

# --- Kaggle auth (works in Colab). If running locally, place kaggle.json at ~/.kaggle/kaggle.json ---
if IN_COLAB:
    if not os.path.exists(os.path.expanduser("~/.kaggle/kaggle.json")):
        print("Please upload your 'kaggle.json' file downloaded from your Kaggle account.")
        uploaded = files.upload()
        if 'kaggle.json' not in uploaded:
            print("\n'kaggle.json' not found. Please restart the runtime and upload the file to proceed.")
        else:
            kaggle_dir = os.path.expanduser('~/.kaggle')
            os.makedirs(kaggle_dir, exist_ok=True)
            with open(os.path.join(kaggle_dir, 'kaggle.json'), 'wb') as f:
                f.write(uploaded['kaggle.json'])
            os.chmod(os.path.join(kaggle_dir, 'kaggle.json'), 0o600)
            print("\nKaggle API credentials configured successfully.")
    else:
        print("Kaggle API credentials already exist.")
else:
    print("Not running in Colab. Make sure kaggle.json is available at ~/.kaggle/kaggle.json if you want to use Kaggle API.")

# ==============================================================================
#                      PART 1: SETUP AND IMPORTS
# ==============================================================================
print("\nPART 1: Installing (if needed) dependencies and importing modules...")

# Install packages if running in Colab or if missing
def ensure_package(pkg_name):
    try:
        __import__(pkg_name)
    except Exception:
        print(f"Installing {pkg_name}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg_name])

# The following packages are required for the notebook
for p in [
    'kagglehub',  # helper used in original code; optional if you have another way to download from Kaggle
    'tensorflow',
    'plotly',
    'seaborn',
    'ipywidgets',
    'opencv-python-headless',
    'Pillow'
]:
    ensure_package(p)

# Now imports
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import EfficientNetV2B2, efficientnet_v2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Conv2D, SeparableConv2D, DepthwiseConv2D

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output
except Exception:
    widgets = None

from PIL import Image
import kagglehub

# --- Centralized Configuration ---
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 30  # EarlyStopping will likely stop earlier
MODEL_SAVE_PATH = 'best_brain_tumor_model_efficient.h5'  # HDF5
print("Setup complete.")

# ==============================================================================
#                  PART 2: DATA DOWNLOAD / PREP (ADJUST IF NEEDED)
# ==============================================================================
print("\nPART 2: Downloading/extracting dataset and preparing data pipeline...")

# NOTE: kagglehub.dataset_download may return either a path to a zip file or a directory.
try:
    dataset_path = kagglehub.dataset_download("masoudnickparvar/brain-tumor-mri-dataset")
except Exception as e:
    print("Warning: kagglehub.download failed or is unavailable. If you already have the dataset, set 'data_source_dir' manually.")
    dataset_path = None

DATA_DIR_BASE = "brain-tumor-mri-dataset"

if dataset_path and os.path.isfile(dataset_path):
    print(f"Dataset downloaded as a zip file. Extracting to '{DATA_DIR_BASE}'...")
    with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
        zip_ref.extractall(DATA_DIR_BASE)
    data_source_dir = DATA_DIR_BASE
elif dataset_path and os.path.isdir(dataset_path):
    print(f"Dataset found as a directory at: {dataset_path}")
    data_source_dir = dataset_path
else:
    # User must ensure they have the dataset locally at DATA_DIR_BASE
    if os.path.isdir(DATA_DIR_BASE):
        data_source_dir = DATA_DIR_BASE
    else:
        raise FileNotFoundError(
            f"Could not find dataset automatically. Please download the dataset and place/extract it at: {DATA_DIR_BASE}"
        )

training_dir = os.path.join(data_source_dir, "Training")
testing_dir = os.path.join(data_source_dir, "Testing")

image_paths = glob.glob(os.path.join(training_dir, "*", "*.jpg"))
class_names = sorted([d for d in os.listdir(training_dir) if os.path.isdir(os.path.join(training_dir, d))])
label_to_index = {name: i for i, name in enumerate(class_names)}
labels = [label_to_index[os.path.basename(os.path.dirname(p))] for p in image_paths]

print(f"\nFound {len(image_paths)} training images belonging to {len(class_names)} classes.")
print("Class Names:", class_names)

train_paths, val_paths, train_labels, val_labels = train_test_split(
    image_paths, labels, test_size=0.2, random_state=42, stratify=labels
)

test_image_paths = glob.glob(os.path.join(testing_dir, "*", "*.jpg"))
if len(test_image_paths) == 0:
    print("Warning: No test images found in Testing/. Proceeding without test set.")
    test_labels = []
else:
    test_labels = [label_to_index[os.path.basename(os.path.dirname(p))] for p in test_image_paths]

# --- TF Data pipeline ---
print("Building tf.data pipelines...")

def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    return image, label


def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    return image, label


def build_dataset(paths, labels, is_training=True):
    AUTOTUNE = tf.data.AUTOTUNE
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    ds = ds.map(lambda p, l: load_and_preprocess_image(p, l), num_parallel_calls=AUTOTUNE)
    if is_training:
        ds = ds.cache()
        ds = ds.shuffle(buffer_size=len(paths))
        ds = ds.map(lambda x, y: augment(x, y), num_parallel_calls=AUTOTUNE)
    ds = ds.batch(BATCH_SIZE)
    ds = ds.map(lambda x, y: (efficientnet_v2.preprocess_input(x), tf.one_hot(y, depth=len(class_names))),
                num_parallel_calls=AUTOTUNE)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

train_ds = build_dataset(train_paths, train_labels)
val_ds = build_dataset(val_paths, val_labels, is_training=False)
if len(test_image_paths) > 0:
    test_ds = build_dataset(test_image_paths, test_labels, is_training=False)
else:
    test_ds = None
print("Data pipeline is ready.")

# ==============================================================================
#         PART 3: MODEL BUILDING, TRAINING, & EVALUATION (CORRECTED)
# ==============================================================================
print("\nPART 3: Building and training the model...")


def build_model_architecture(num_classes):
    base_model = EfficientNetV2B2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    base_model.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

model = build_model_architecture(len(class_names))

model.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
callbacks = [
    ModelCheckpoint(MODEL_SAVE_PATH, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6, verbose=1)
]

print("--- Starting Feature Extraction ---")
history = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds, callbacks=callbacks, verbose=1)

print("\n--- Starting Fine-Tuning ---")
model.trainable = True
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
history_fine = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds, callbacks=callbacks, verbose=1)

print("\n--- Evaluating Final Model on Test Set ---")
# Load best saved model
if os.path.exists(MODEL_SAVE_PATH):
    best_model = load_model(MODEL_SAVE_PATH)
    best_model.compile(loss='categorical_crossentropy', metrics=['accuracy'])
    if test_ds is not None:
        test_loss, test_acc = best_model.evaluate(test_ds)
        print(f"\nFinal Test Accuracy: {test_acc*100:.2f}%")

        y_pred_probs = best_model.predict(test_ds)
        y_pred = np.argmax(y_pred_probs, axis=1)
        y_true = np.array(test_labels)

        print("\nClassification Report:")
        print(classification_report(y_true, y_pred, target_names=class_names))

        cm = confusion_matrix(y_true, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
        plt.title('Confusion Matrix')
        plt.ylabel('Actual')
        plt.xlabel('Predicted')
        plt.show()
    else:
        print("No test set available to evaluate.")
else:
    print(f"Model file {MODEL_SAVE_PATH} not found. Skipping final test evaluation.")

# ==============================================================================
#              PART 4: INTERACTIVE UI APPLICATION (ROBUST FIX)
# ==============================================================================
print("\nPART 4: Launching interactive analysis application (widgets)...")


def get_suggestions(predicted_class):
    suggestions = {
        'glioma': ("**About Glioma:** A common tumor originating in the glial cells of the brain. It can be malignant.\n\n"
                   "**Next Steps:**\n"
                   "1. **Consult a Neuro-oncologist:** Immediate specialist consultation is crucial.\n"
                   "2. **Advanced Imaging:** Further tests like MRI with contrast or biopsy are often needed.\n"
                   "3. **Treatment:** Typically includes surgery, radiation, and chemotherapy."),
        'meningioma': ("**About Meningioma:** A tumor from the meninges (brain's protective membranes). Most are benign.\n\n"
                       "**Next Steps:**\n"
                       "1. **Consult a Neurologist/Neurosurgeon:** To discuss the findings.\n"
                       "2. **Observation:** A 'watch-and-wait' approach may be recommended for small, asymptomatic tumors.\n"
                       "3. **Treatment:** Surgery or radiation may be used if it's large or causes symptoms."),
        'pituitary': ("**About Pituitary Tumor:** A growth in the pituitary gland. Most are benign but can affect hormones.\n\n"
                      "**Next Steps:**\n"
                      "1. **Consult an Endocrinologist:** A hormone specialist is key.\n"
                      "2. **Hormone Level Testing:** Blood and urine tests are needed.\n"
                      "3. **Treatment:** May include medication, surgery, or radiation."),
        'notumor': ("**No Tumor Detected:** The model did not find signs of a tumor.\n\n"
                    "**Recommendations:**\n"
                    "- This is a positive sign, but this is not a medical diagnosis.\n"
                    "- Always consult a qualified radiologist for a definitive diagnosis.\n"
                    "- Continue with regular health check-ups.")
    }
    return suggestions.get(predicted_class, "No information available.")


class PredictionPipeline:
    def __init__(self, model_path, class_names):
        self.class_names = class_names
        # Load model
        if not os.path.exists(model_path):
            raise FileNotFoundError(f"Model file '{model_path}' not found. Please train/save the model first.")
        self.model = load_model(model_path)

        # Try to find the last convolutional layer robustly
        conv_types = (Conv2D, SeparableConv2D, DepthwiseConv2D)
        last_conv_layer_name = None

        # First pass: check explicit conv-type layers (preferred)
        for layer in reversed(self.model.layers):
            if isinstance(layer, conv_types):
                last_conv_layer_name = layer.name
                break

        # Second pass: fallback to checking output shapes if none matched above
        if last_conv_layer_name is None:
            for layer in reversed(self.model.layers):
                out_shape = None
                if hasattr(layer, 'output_shape'):
                    try:
                        out_shape = layer.output_shape
                    except Exception:
                        out_shape = None
                elif hasattr(layer, 'output'):
                    try:
                        # layer.output may be a Tensor
                        out_shape = getattr(layer.output, 'shape', None)
                    except Exception:
                        out_shape = None
                if out_shape is None:
                    continue
                try:
                    if len(out_shape) == 4:
                        last_conv_layer_name = layer.name
                        break
                except Exception:
                    continue

        if last_conv_layer_name is None:
            raise ValueError("Could not find a valid convolutional layer for Grad-CAM. Make sure the model contains Conv2D-like layers.")

        self.last_conv_layer_name = last_conv_layer_name
        # grad_model will give us conv outputs and final predictions
        self.grad_model = Model([self.model.inputs], [self.model.get_layer(last_conv_layer_name).output, self.model.output])

    def predict_and_visualize(self, img_bytes):
        # Preprocess
        img = Image.open(io.BytesIO(img_bytes)).convert('RGB').resize((IMG_SIZE, IMG_SIZE))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        preprocessed_img = efficientnet_v2.preprocess_input(np.expand_dims(img_array, axis=0))

        # Predict
        prediction = self.model.predict(preprocessed_img, verbose=0)
        predicted_class_index = int(np.argmax(prediction, axis=1)[0])
        predicted_class_name = self.class_names[predicted_class_index]
        confidence = float(np.max(prediction) * 100)

        # Grad-CAM
        with tf.GradientTape() as tape:
            conv_outputs, preds = self.grad_model(preprocessed_img)
            # Ensure the conv outputs are being watched for gradient computation
            tape.watch(conv_outputs)
            loss = preds[:, predicted_class_index]
        grads = tape.gradient(loss, conv_outputs)
        if grads is None:
            # Fallback: gradients couldn't be computed
            heatmap = np.zeros((conv_outputs.shape[1], conv_outputs.shape[2]))
        else:
            pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
            # Compute weighted combination of feature maps
            conv_outputs_np = conv_outputs[0].numpy()
            pooled_grads_np = pooled_grads.numpy()
            heatmap = np.zeros(conv_outputs_np.shape[:2], dtype=np.float32)
            for i in range(conv_outputs_np.shape[-1]):
                heatmap += conv_outputs_np[:, :, i] * pooled_grads_np[i]

            # ReLU and normalize
            heatmap = np.maximum(heatmap, 0)
            if np.max(heatmap) != 0:
                heatmap /= np.max(heatmap)

        # Superimpose heatmap on original image
        original_img_cv = np.array(img).astype(np.uint8)
        heatmap_resized = cv2.resize(heatmap, (original_img_cv.shape[1], original_img_cv.shape[0]))
        heatmap_uint8 = np.uint8(255 * heatmap_resized)
        heatmap_color = cv2.applyColorMap(heatmap_uint8, cv2.COLORMAP_JET)
        # Convert color map from BGR to RGB (cv2 returns BGR)
        heatmap_color = cv2.cvtColor(heatmap_color, cv2.COLOR_BGR2RGB)
        superimposed_img = cv2.addWeighted(original_img_cv, 0.6, heatmap_color, 0.4, 0)

        return predicted_class_name, confidence, prediction.flatten(), superimposed_img


# --- Instantiate Pipeline and UI Widgets (if ipywidgets available) ---
try:
    pipeline = PredictionPipeline(MODEL_SAVE_PATH, class_names)
    print(f"Grad-CAM will use layer: {pipeline.last_conv_layer_name}")
except Exception as e:
    print("Warning: Could not instantiate PredictionPipeline:", e)
    pipeline = None

if widgets is not None and pipeline is not None:
    uploader = widgets.FileUpload(accept='image/*', description='Upload MRI', button_style='primary')
    output_area = widgets.Output()

    # helper to convert **bold** to <b>...</b> safely (handles matching pairs)
    def md_bold_to_html(text):
        return re.sub(r"\*\*(.*?)\*\*", r"<b>\1</b>", text, flags=re.S)

    def on_upload_change(change):
        with output_area:
            clear_output(wait=True)
            if not change['new']:
                return
            # The FileUpload widget stores files as a dict of name->metadata. Grab first file.
            first_key = list(change['new'].keys())[0]
            file_info = change['new'][first_key]
            img_bytes = file_info['content']

            predicted_class, confidence, scores, gradcam_img = pipeline.predict_and_visualize(img_bytes)

            display(widgets.HTML(f"""
                <h3>Prediction: <span style='color:#007bff;'>{predicted_class.upper()}</span></h3>
                <h4>Confidence: {confidence:.2f}%</h4>"""))

            _, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
            ax1.imshow(Image.open(io.BytesIO(img_bytes)))
            ax1.set_title("Uploaded MRI")
            ax1.axis('off')
            ax2.imshow(gradcam_img)
            ax2.set_title("Grad-CAM Heatmap")
            ax2.axis('off')
            plt.show()

            df_pred = pd.DataFrame({'Class': class_names, 'Confidence': (scores * 100).astype(float)})
            fig = px.bar(df_pred, x='Class', y='Confidence', text=df_pred['Confidence'].apply(lambda x: f'{x:.2f}%'),
                         title="Prediction Confidence Scores")
            fig.show()

            suggestions_html = md_bold_to_html(get_suggestions(predicted_class)).replace('\n', '<br>')
            display(widgets.HTML(f"""
                <div style='border-top: 2px solid #007bff; margin-top: 20px; padding-top: 10px;'>
                <h3>Information & Recommendations</h3>
                <p>{suggestions_html}</p></div>"""))

    uploader.observe(on_upload_change, names='value')

    display(widgets.VBox([
        widgets.HTML("<h2>üß† Brain Tumor Classification & Analysis</h2>"
                     "<p>Upload a brain MRI scan to get an AI-powered analysis. The model will run and display results below.</p>"
                     "<p style='color:red;'><b>Disclaimer:</b> This is an educational tool, not a substitute for professional medical diagnosis.</p>"),
        uploader,
        output_area
    ]))
else:
    print("ipywidgets not available or pipeline couldn't be created. Skipping interactive UI.")

print('\nAll done. If you want any further tweaks (e.g., different Grad-CAM normalization, other visualization styles, or saving the superimposed images), tell me what you prefer.')


PART 0: Authenticating with Kaggle API...
Please upload your 'kaggle.json' file downloaded from your Kaggle account.


Saving kaggle.json to kaggle.json

Kaggle API credentials configured successfully.

PART 1: Installing (if needed) dependencies and importing modules...
Installing opencv-python-headless...
Installing Pillow...
Setup complete.

PART 2: Downloading/extracting dataset and preparing data pipeline...
Using Colab cache for faster access to the 'brain-tumor-mri-dataset' dataset.
Dataset found as a directory at: /kaggle/input/brain-tumor-mri-dataset

Found 5712 training images belonging to 4 classes.
Class Names: ['glioma', 'meningioma', 'notumor', 'pituitary']
Building tf.data pipelines...
Data pipeline is ready.

PART 3: Building and training the model...
35839040/35839040 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 1s 0us/step
--- Starting Feature Extraction ---
Epoch 1/30
143/143 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 0s 204ms/step - accuracy: 0.5963 - loss: 0.9661
Epoch 1: val_accuracy improved from -inf to 0.82765, saving model to best_brain_tumor_

143/143 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 112s 375ms/step - accuracy: 0.5971 - loss: 0.9645 - val_accuracy: 0.8276 - val_loss: 0.4420 - learning_rate: 0.0010
Epoch 2/30
143/143 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 0s 48ms/step - accuracy: 0.8128 - loss: 0.5102
Epoch 2: val_accuracy improved from 0.82765 to 0.87402, saving model to best_brain_tumor_model_efficient.h5


143/143 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 10s 72ms/step - accuracy: 0.8128 - loss: 0.5101 - val_accuracy: 0.8740 - val_loss: 0.3535 - learning_rate: 0.0010
Epoch 3/30
142/143 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 0s 49ms/step - accuracy: 0.8419 - loss: 0.4317
Epoch 3: val_accuracy improved from 0.87402 to 0.87927, saving model to best_brain_tumor_model_efficient.h5


143/143 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 10s 67ms/step - accuracy: 0.8418 - loss: 0.4317 - val_accuracy: 0.8793 - val_loss: 0.3232 - learning_rate: 0.0010
Epoch 4/30
142/143 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 0s 49ms/step - accuracy: 0.8530 - loss: 0.4085
Epoch 4: val_accuracy improved from 0.87927 to 0.88626, saving model to best_brain_tumor_model_efficient.h5


...

143/143 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 11s 69ms/step - accuracy: 0.8809 - loss: 0.3114 - val_accuracy: 0.9283 - val_loss: 0.2073 - learning_rate: 0.0010
Epoch 23/30
142/143 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 0s 51ms/step - accuracy: 0.8807 - loss: 0.3230
Epoch 23: val_accuracy did not improve from 0.92826
143/143 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 9s 64ms/step - accuracy: 0.8808 - loss: 0.3229 - val_accuracy: 0.9125 - val_loss: 0.2285 - learning_rate: 0.0010
Epoch 24/30
142/143 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 0s 50ms/step - accuracy: 0.8627 - loss: 0.3565
Epoch 24: val_accuracy did not improve from 0.92826

Epoch 24: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
143/143 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 9s 65ms/step - accuracy: 0.8630 - loss: 0.3560 - val_accuracy: 0.9073 - val_loss: 0.2266 - learning_rate: 0.0010
Epoch 25/30
14

41/41 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 26s 384ms/step - accuracy: 0.9288 - loss: 0.1836

Final Test Accuracy: 88.48%
41/41 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 18s 258ms/step

Classification Report:
              precision    recall  f1-score   support

      glioma       0.93      0.79      0.85       300
  meningioma       0.77      0.80      0.78       306
     notumor       0.92      0.98      0.94       405
   pituitary       0.93      0.94      0.94       300

    accuracy                           0.88      1311
   macro avg       0.88      0.88      0.88      1311
weighted avg       0.89      0.88      0.88      1311




PART 4: Launching interactive analysis application (widgets)...


Grad-CAM will use layer: top_conv



All done. If you want any further tweaks (e.g., different Grad-CAM normalization, other visualization styles, or saving the superimposed images), tell me what you prefer.


In [ ]:
# ==============================================================================
#      FULL FIXED & REFINED: Brain Tumor Classification (Colab-ready)
# ==============================================================================
# This script should be pasted into a Google Colab cell or saved as a .py file
# to be run in a suitable environment.
#
# Main Refinements:
#  - Deeper and more robust model head for improved accuracy and reduced false positives.
#  - Increased training epochs with early stopping for more thorough training.
#  - Maintained robust Grad-CAM layer detection and safe filesystem operations.
#  - Improved comments and structure for better readability and maintainability.
#

print("PART 0: Authenticating with Kaggle API...")

import os
import sys
import re
import zipfile
import glob
import io
import shutil
import subprocess

# Colab-specific utilities (if running in Colab)
try:
    from google.colab import files
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# --- Kaggle auth (works in Colab). If running locally, place kaggle.json at ~/.kaggle/kaggle.json ---
if IN_COLAB:
    if not os.path.exists(os.path.expanduser("~/.kaggle/kaggle.json")):
        print("Please upload your 'kaggle.json' file downloaded from your Kaggle account.")
        uploaded = files.upload()
        if 'kaggle.json' not in uploaded:
            print("\n'kaggle.json' not found. Please restart the runtime and upload the file to proceed.")
        else:
            kaggle_dir = os.path.expanduser('~/.kaggle')
            os.makedirs(kaggle_dir, exist_ok=True)
            with open(os.path.join(kaggle_dir, 'kaggle.json'), 'wb') as f:
                f.write(uploaded['kaggle.json'])
            os.chmod(os.path.join(kaggle_dir, 'kaggle.json'), 0o600)
            print("\nKagle API credentials configured successfully.")
    else:
        print("Kaggle API credentials already exist.")
else:
    print("Not running in Colab. Make sure kaggle.json is available at ~/.kaggle/kaggle.json if you want to use Kaggle API.")

# ==============================================================================
#                      PART 1: SETUP AND IMPORTS
# ==============================================================================
print("\nPART 1: Installing (if needed) dependencies and importing modules...")

# Install packages if running in Colab or if missing
def ensure_package(pkg_name):
    try:
        __import__(pkg_name)
    except ImportError:
        print(f"Installing {pkg_name}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg_name])

# The following packages are required for the notebook
for p in [
    'kagglehub',
    'tensorflow',
    'plotly',
    'seaborn',
    'ipywidgets',
    'opencv-python-headless',
    'Pillow'
]:
    ensure_package(p)

# Now imports
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.applications import EfficientNetV2B2, efficientnet_v2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Conv2D, SeparableConv2D, DepthwiseConv2D

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output
except ImportError:
    widgets = None

from PIL import Image
import kagglehub

# --- Centralized Configuration ---
IMG_SIZE = 224
BATCH_SIZE = 32
# Increased epochs for more thorough training, with EarlyStopping to prevent overfitting
EPOCHS = 50
MODEL_SAVE_PATH = 'best_brain_tumor_model_efficient_refined.h5'  # HDF5
print("Setup complete.")

# ==============================================================================
#                  PART 2: DATA DOWNLOAD / PREP (ADJUST IF NEEDED)
# ==============================================================================
print("\nPART 2: Downloading/extracting dataset and preparing data pipeline...")

try:
    dataset_path = kagglehub.dataset_download("masoudnickparvar/brain-tumor-mri-dataset")
except Exception as e:
    print("Warning: kagglehub.download failed or is unavailable. If you already have the dataset, set 'data_source_dir' manually.")
    dataset_path = None

DATA_DIR_BASE = "brain-tumor-mri-dataset"

if dataset_path and os.path.isfile(dataset_path):
    print(f"Dataset downloaded as a zip file. Extracting to '{DATA_DIR_BASE}'...")
    with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
        zip_ref.extractall(DATA_DIR_BASE)
    data_source_dir = DATA_DIR_BASE
elif dataset_path and os.path.isdir(dataset_path):
    print(f"Dataset found as a directory at: {dataset_path}")
    data_source_dir = dataset_path
else:
    if os.path.isdir(DATA_DIR_BASE):
        data_source_dir = DATA_DIR_BASE
    else:
        raise FileNotFoundError(
            f"Could not find dataset automatically. Please download the dataset and place/extract it at: {DATA_DIR_BASE}"
        )

training_dir = os.path.join(data_source_dir, "Training")
testing_dir = os.path.join(data_source_dir, "Testing")

image_paths = glob.glob(os.path.join(training_dir, "*", "*.jpg"))
class_names = sorted([d for d in os.listdir(training_dir) if os.path.isdir(os.path.join(training_dir, d))])
label_to_index = {name: i for i, name in enumerate(class_names)}
labels = [label_to_index[os.path.basename(os.path.dirname(p))] for p in image_paths]

print(f"\nFound {len(image_paths)} training images belonging to {len(class_names)} classes.")
print("Class Names:", class_names)

train_paths, val_paths, train_labels, val_labels = train_test_split(
    image_paths, labels, test_size=0.2, random_state=42, stratify=labels
)

test_image_paths = glob.glob(os.path.join(testing_dir, "*", "*.jpg"))
if len(test_image_paths) == 0:
    print("Warning: No test images found in Testing/. Proceeding without test set.")
    test_labels = []
else:
    test_labels = [label_to_index[os.path.basename(os.path.dirname(p))] for p in test_image_paths]

# --- TF Data pipeline ---
print("Building tf.data pipelines...")

def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    return image, label

def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    return image, label

def build_dataset(paths, labels, is_training=True):
    AUTOTUNE = tf.data.AUTOTUNE
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    ds = ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
    if is_training:
        ds = ds.cache()
        ds = ds.shuffle(buffer_size=len(paths))
        ds = ds.map(augment, num_parallel_calls=AUTOTUNE)
    ds = ds.batch(BATCH_SIZE)
    ds = ds.map(lambda x, y: (efficientnet_v2.preprocess_input(x), tf.one_hot(y, depth=len(class_names))),
                num_parallel_calls=AUTOTUNE)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

train_ds = build_dataset(train_paths, train_labels)
val_ds = build_dataset(val_paths, val_labels, is_training=False)
if len(test_image_paths) > 0:
    test_ds = build_dataset(test_image_paths, test_labels, is_training=False)
else:
    test_ds = None
print("Data pipeline is ready.")

# ==============================================================================
#         PART 3: MODEL BUILDING, TRAINING, & EVALUATION (REFINED)
# ==============================================================================
print("\nPART 3: Building and training the refined model...")

def build_refined_model_architecture(num_classes):
    base_model = EfficientNetV2B2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    base_model.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    # Adding more layers for a more complex head
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.7)(x) # Increased dropout for better regularization

    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

model = build_refined_model_architecture(len(class_names))

model.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
callbacks = [
    ModelCheckpoint(MODEL_SAVE_PATH, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1),
    EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True, verbose=1), # Increased patience
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6, verbose=1)
]

print("--- Starting Feature Extraction ---")
history = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds, callbacks=callbacks, verbose=1)

print("\n--- Starting Fine-Tuning ---")
model.trainable = True
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
history_fine = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds, callbacks=callbacks, verbose=1)

print("\n--- Evaluating Final Model on Test Set ---")
if os.path.exists(MODEL_SAVE_PATH):
    best_model = load_model(MODEL_SAVE_PATH)
    best_model.compile(loss='categorical_crossentropy', metrics=['accuracy'])
    if test_ds is not None:
        test_loss, test_acc = best_model.evaluate(test_ds)
        print(f"\nFinal Test Accuracy: {test_acc*100:.2f}%")

        y_pred_probs = best_model.predict(test_ds)
        y_pred = np.argmax(y_pred_probs, axis=1)
        y_true = np.array(test_labels)

        print("\nClassification Report:")
        print(classification_report(y_true, y_pred, target_names=class_names))

        cm = confusion_matrix(y_true, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
        plt.title('Confusion Matrix')
        plt.ylabel('Actual')
        plt.xlabel('Predicted')
        plt.show()
    else:
        print("No test set available to evaluate.")
else:
    print(f"Model file {MODEL_SAVE_PATH} not found. Skipping final test evaluation.")

# ==============================================================================
#              PART 4: INTERACTIVE UI APPLICATION (ROBUST FIX)
# ==============================================================================
print("\nPART 4: Launching interactive analysis application (widgets)...")

def get_suggestions(predicted_class):
    suggestions = {
        'glioma': ("**About Glioma:** A common tumor originating in the glial cells of the brain. It can be malignant.\n\n"
                   "**Next Steps:**\n"
                   "1. **Consult a Neuro-oncologist:** Immediate specialist consultation is crucial.\n"
                   "2. **Advanced Imaging:** Further tests like MRI with contrast or biopsy are often needed.\n"
                   "3. **Treatment:** Typically includes surgery, radiation, and chemotherapy."),
        'meningioma': ("**About Meningioma:** A tumor from the meninges (brain's protective membranes). Most are benign.\n\n"
                       "**Next Steps:**\n"
                       "1. **Consult a Neurologist/Neurosurgeon:** To discuss the findings.\n"
                       "2. **Observation:** A 'watch-and-wait' approach may be recommended for small, asymptomatic tumors.\n"
                       "3. **Treatment:** Surgery or radiation may be used if it's large or causes symptoms."),
        'pituitary': ("**About Pituitary Tumor:** A growth in the pituitary gland. Most are benign but can affect hormones.\n\n"
                      "**Next Steps:**\n"
                      "1. **Consult an Endocrinologist:** A hormone specialist is key.\n"
                      "2. **Hormone Level Testing:** Blood and urine tests are needed.\n"
                      "3. **Treatment:** May include medication, surgery, or radiation."),
        'notumor': ("**No Tumor Detected:** The model did not find signs of a tumor.\n\n"
                    "**Recommendations:**\n"
                    "- This is a positive sign, but this is not a medical diagnosis.\n"
                    "- Always consult a qualified radiologist for a definitive diagnosis.\n"
                    "- Continue with regular health check-ups.")
    }
    return suggestions.get(predicted_class, "No information available.")

class PredictionPipeline:
    def __init__(self, model_path, class_names):
        self.class_names = class_names
        if not os.path.exists(model_path):
            raise FileNotFoundError(f"Model file '{model_path}' not found. Please train/save the model first.")
        self.model = load_model(model_path)

        conv_types = (Conv2D, SeparableConv2D, DepthwiseConv2D)
        last_conv_layer_name = None

        for layer in reversed(self.model.layers):
            if isinstance(layer, conv_types):
                last_conv_layer_name = layer.name
                break

        if last_conv_layer_name is None:
            for layer in reversed(self.model.layers):
                if hasattr(layer, 'output_shape') and len(layer.output_shape) == 4:
                    last_conv_layer_name = layer.name
                    break

        if last_conv_layer_name is None:
            raise ValueError("Could not find a valid convolutional layer for Grad-CAM.")

        self.last_conv_layer_name = last_conv_layer_name
        self.grad_model = Model(
            [self.model.inputs], [self.model.get_layer(last_conv_layer_name).output, self.model.output]
        )

    def predict_and_visualize(self, img_bytes):
        img = Image.open(io.BytesIO(img_bytes)).convert('RGB').resize((IMG_SIZE, IMG_SIZE))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        preprocessed_img = efficientnet_v2.preprocess_input(np.expand_dims(img_array, axis=0))

        prediction = self.model.predict(preprocessed_img, verbose=0)
        predicted_class_index = int(np.argmax(prediction, axis=1)[0])
        predicted_class_name = self.class_names[predicted_class_index]
        confidence = float(np.max(prediction) * 100)

        with tf.GradientTape() as tape:
            conv_outputs, preds = self.grad_model(preprocessed_img)
            tape.watch(conv_outputs)
            loss = preds[:, predicted_class_index]

        grads = tape.gradient(loss, conv_outputs)
        if grads is None:
            heatmap = np.zeros((conv_outputs.shape[1], conv_outputs.shape[2]))
        else:
            pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
            conv_outputs_np = conv_outputs[0].numpy()
            pooled_grads_np = pooled_grads.numpy()
            heatmap = np.dot(conv_outputs_np, pooled_grads_np)

            heatmap = np.maximum(heatmap, 0)
            if np.max(heatmap) != 0:
                heatmap /= np.max(heatmap)

        original_img_cv = np.array(img).astype(np.uint8)
        heatmap_resized = cv2.resize(heatmap, (original_img_cv.shape[1], original_img_cv.shape[0]))
        heatmap_uint8 = np.uint8(255 * heatmap_resized)
        heatmap_color = cv2.applyColorMap(heatmap_uint8, cv2.COLORMAP_JET)
        heatmap_color = cv2.cvtColor(heatmap_color, cv2.COLOR_BGR2RGB)
        superimposed_img = cv2.addWeighted(original_img_cv, 0.6, heatmap_color, 0.4, 0)

        return predicted_class_name, confidence, prediction.flatten(), superimposed_img

try:
    pipeline = PredictionPipeline(MODEL_SAVE_PATH, class_names)
    print(f"Grad-CAM will use layer: {pipeline.last_conv_layer_name}")
except Exception as e:
    print("Warning: Could not instantiate PredictionPipeline:", e)
    pipeline = None

if widgets is not None and pipeline is not None:
    uploader = widgets.FileUpload(accept='image/*', description='Upload MRI', button_style='primary')
    output_area = widgets.Output()

    def md_bold_to_html(text):
        return re.sub(r"\*\*(.*?)\*\*", r"<b>\1</b>", text, flags=re.S)

    def on_upload_change(change):
        with output_area:
            clear_output(wait=True)
            if not change['new']:
                return

            first_key = list(change['new'].keys())[0]
            file_info = change['new'][first_key]
            img_bytes = file_info['content']

            predicted_class, confidence, scores, gradcam_img = pipeline.predict_and_visualize(img_bytes)

            display(widgets.HTML(f"""
                <h3>Prediction: <span style='color:#007bff;'>{predicted_class.upper()}</span></h3>
                <h4>Confidence: {confidence:.2f}%</h4>"""))

            _, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
            ax1.imshow(Image.open(io.BytesIO(img_bytes)))
            ax1.set_title("Uploaded MRI")
            ax1.axis('off')
            ax2.imshow(gradcam_img)
            ax2.set_title("Grad-CAM Heatmap")
            ax2.axis('off')
            plt.show()

            df_pred = pd.DataFrame({'Class': class_names, 'Confidence': (scores * 100).astype(float)})
            fig = px.bar(df_pred, x='Class', y='Confidence', text=df_pred['Confidence'].apply(lambda x: f'{x:.2f}%'),
                         title="Prediction Confidence Scores")
            fig.show()

            suggestions_html = md_bold_to_html(get_suggestions(predicted_class)).replace('\n', '<br>')
            display(widgets.HTML(f"""
                <div style='border-top: 2px solid #007bff; margin-top: 20px; padding-top: 10px;'>
                <h3>Information & Recommendations</h3>
                <p>{suggestions_html}</p></div>"""))

    uploader.observe(on_upload_change, names='value')

    display(widgets.VBox([
        widgets.HTML("<h2>üß† Brain Tumor Classification & Analysis</h2>"
                     "<p>Upload a brain MRI scan to get an AI-powered analysis. The model will run and display results below.</p>"
                     "<p style='color:red;'><b>Disclaimer:</b> This is an educational tool, not a substitute for professional medical diagnosis.</p>"),
        uploader,
        output_area
    ]))
else:
    print("ipywidgets not available or pipeline couldn't be created. Skipping interactive UI.")

print('\nAll done. This refined model should provide better performance. For Streamlit, you can adapt the `PredictionPipeline` class and the logic within `on_upload_change` to fit into your Streamlit application script.')

PART 0: Authenticating with Kaggle API...
Please upload your 'kaggle.json' file downloaded from your Kaggle account.


Saving kaggle.json to kaggle.json

Kagle API credentials configured successfully.

PART 1: Installing (if needed) dependencies and importing modules...
Installing opencv-python-headless...
Installing Pillow...
Setup complete.

PART 2: Downloading/extracting dataset and preparing data pipeline...
Using Colab cache for faster access to the 'brain-tumor-mri-dataset' dataset.
Dataset found as a directory at: /kaggle/input/brain-tumor-mri-dataset

Found 5712 training images belonging to 4 classes.
Class Names: ['glioma', 'meningioma', 'notumor', 'pituitary']
Building tf.data pipelines...
Data pipeline is ready.

PART 3: Building and training the refined model...
35839040/35839040 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 3s 0us/step
--- Starting Feature Extraction ---
Epoch 1/50
143/143 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 0s 199ms/step - accuracy: 0.7105 - loss: 1.0195
Epoch 1: val_accuracy improved from -inf to 0.87927, saving model to best_brain

143/143 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 109s 382ms/step - accuracy: 0.7109 - loss: 1.0177 - val_accuracy: 0.8793 - val_loss: 0.3012 - learning_rate: 0.0010
Epoch 2/50
142/143 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 0s 46ms/step - accuracy: 0.8369 - loss: 0.4798
Epoch 2: val_accuracy improved from 0.87927 to 0.89501, saving model to best_brain_tumor_model_efficient_refined.h5


143/143 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 9s 64ms/step - accuracy: 0.8370 - loss: 0.4795 - val_accuracy: 0.8950 - val_loss: 0.2614 - learning_rate: 0.0010
Epoch 3/50
143/143 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 0s 47ms/step - accuracy: 0.8549 - loss: 0.3964
Epoch 3: val_accuracy improved from 0.89501 to 0.90726, saving model to best_brain_tumor_model_efficient_refined.h5


143/143 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 9s 64ms/step - accuracy: 0.8549 - loss: 0.3964 - val_accuracy: 0.9073 - val_loss: 0.2540 - learning_rate: 0.0010
Epoch 4/50
142/143 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 0s 47ms/step - accuracy: 0.8693 - loss: 0.3477
Epoch 4: val_accuracy improved from 0.90726 to 0.91426, saving model to best_brain_tumor_model_efficient_refined.h5


...

143/143 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 9s 65ms/step - accuracy: 0.9333 - loss: 0.1861 - val_accuracy: 0.9571 - val_loss: 0.1212 - learning_rate: 2.0000e-04
Epoch 32/50
142/143 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 0s 48ms/step - accuracy: 0.9282 - loss: 0.1851
Epoch 32: val_accuracy did not improve from 0.95713

Epoch 32: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.
143/143 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 10s 66ms/step - accuracy: 0.9283 - loss: 0.1849 - val_accuracy: 0.9519 - val_loss: 0.1234 - learning_rate: 2.0000e-04
Epoch 33/50
142/143 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 0s 46ms/step - accuracy: 0.9360 - loss: 0.1752
Epoch 33: val_accuracy did not improve from 0.95713
143/143 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 9s 66ms/step - accuracy: 0.9360 - loss: 0.1752 - val_accuracy: 0.9563 - val_loss: 0.1197 - learning_rate: 4.0000e-05
Ep

41/41 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 25s 355ms/step - accuracy: 0.9740 - loss: 0.0724

Final Test Accuracy: 94.81%
41/41 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 16s 240ms/step

Classification Report:
              precision    recall  f1-score   support

      glioma       0.98      0.87      0.92       300
  meningioma       0.88      0.93      0.90       306
     notumor       0.98      0.99      0.99       405
   pituitary       0.95      0.98      0.97       300

    accuracy                           0.95      1311
   macro avg       0.95      0.94      0.94      1311
weighted avg       0.95      0.95      0.95      1311




PART 4: Launching interactive analysis application (widgets)...


Grad-CAM will use layer: top_conv



All done. This refined model should provide better performance. For Streamlit, you can adapt the `PredictionPipeline` class and the logic within `on_upload_change` to fit into your Streamlit application script.
